### Code to generate a sequenceof samples, based on one hot encoding using recurrent neural networks

In [84]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import tensorflow as tf

In [85]:
def parseMatrixFromFile(fname):
    with open(fname) as f:
        for l in f:
            yield [int(x) for x in l.strip().split()]

In [86]:
rhythm_data = list(parseMatrixFromFile("mus_dataset.txt"))

In [87]:
len(rhythm_data[0])

21

In [88]:
data_y = [e[20] for e in rhythm_data]

In [89]:
data_X = [e[:-1] for e in rhythm_data]

In [90]:
print(data_X[0])
print(data_y[0])
print(rhythm_data[0])

[1, 3, 4, 5, 6, 5, 6, 2, 7, 8, 2, 8, 3, 8, 9, 2, 4, 6, 8, 0]
3
[1, 3, 4, 5, 6, 5, 6, 2, 7, 8, 2, 8, 3, 8, 9, 2, 4, 6, 8, 0, 3]


In [91]:
# reshape X to be [samples, time_steps, features]
X = np.reshape(data_X, (len(data_X), len(data_X[0]), 1))

# one hot encode the output variable
y = np_utils.to_categorical(data_y)

In [92]:
y[0]

array([ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.])

In [93]:
print(np.shape(X))
print(np.shape(y))

(2000, 20, 1)
(2000, 10)


In [95]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [96]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:
with tf.device('/gpu:0'):
    model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

In [ ]:
# load the network weights # from saved checkpoint files
filename = "weights-improvement-19-1.9394.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [103]:
generated_rhythm = []
# Should be a multiple of len(rhythm_data[0])
num_samples = 210 #len(dataX)*len(rhythm_data[0])

start = np.random.randint(0, len(data_X)-1)
pattern = data_X[start]
print("Seed:")
print(pattern)

# generate sequence with a random seed
for i in range(num_samples):
    x = np.reshape(pattern, (1, len(pattern), 1))
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    
    generated_rhythm.append(index)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("Generation Completed!")

Seed:
[5, 6, 8, 2, 2, 3, 5, 3, 5, 7, 3, 7, 8, 9, 2, 5, 7, 8, 0, 1]
Generation Completed!


In [104]:
generated_samples = np.reshape(generated_rhythm, (num_samples//len(rhythm_data[0]),len(rhythm_data[0])))

In [105]:
np.shape(generated_samples)

(10, 21)

### Here generated samples is the output

In [106]:
generated_samples

array([[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9],
       [9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9],
       [9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9],
       [9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9],
       [9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9],
       [9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9],
       [9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9],
       [9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9],
       [9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9],
       [9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]], dtype=int64)

In [113]:
# Saving generated samples to a file.
np.savetxt('mus_generated.txt', generated_samples, delimiter=' ', fmt='%d', newline="\n")